In [1]:
from konlpy.tag import Kkma
import nltk
from nltk.tag import pos_tag
from nltk.corpus import gutenberg
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import regexp_tokenize
from nltk.help import upenn_tagset
from string import punctuation
from nltk import Text
from konlpy.corpus import kolaw, kobill
from konlpy.tag import Kkma, Mecab, Okt, Komoran, Hannanum
import pandas as pd
import numpy as np
import re

In [208]:
similarity_table = pd.DataFrame()

In [209]:
# symbol dictionary
sd = {'½': '이분의일',
 '□': '사각형',
 '＞': '보다크다',
 '㈏': '나',
 '(': '',
 '×': '곱하기',
 ')': '',
 '①': '일번',
 '+': '더하기',
 '˚': '도',
 '⑤': '오번',
 '⅓': '삼분의일',
 '⅔': '삼분의이',
 '¼': '사분의일',
 '′': '',
 '＜': '보다작다',
 '-': '빼기',
 '!': '',
 '④': '사번',
 'π': '파이',
 '㎞': '킬로미터',
 '=': '는',
 ':': '대',
 '~': '',
 '…': '',
 '△': '삼각형',
 '²': '의제곱',
 '②': '이번',
 '㈎': '가',
 '≤': '보다크거나같다',
 '㎝': '센치미터',
 '|': '절대값',
 '㈐': '다',
 '＝': '는',
 '÷': '나누기',
 '⑵': '이번',
 '㎠': '제곱센치미터',
 '¾': '사분의삼',
 '∠': '각',
 '%': '퍼센트',
 '－': '마이너스',
 "'": '',
 '③': '삼번',
 '＋': '더하기',
 '⑴': '일번',
 '√': '루트',
 'cos': '코사인',
 'sin': '사인',
 'tan': '탄젠트'}
sd

{'½': '이분의일',
 '□': '사각형',
 '＞': '보다크다',
 '㈏': '나',
 '(': '',
 '×': '곱하기',
 ')': '',
 '①': '일번',
 '+': '더하기',
 '˚': '도',
 '⑤': '오번',
 '⅓': '삼분의일',
 '⅔': '삼분의이',
 '¼': '사분의일',
 '′': '',
 '＜': '보다작다',
 '-': '빼기',
 '!': '',
 '④': '사번',
 'π': '파이',
 '㎞': '킬로미터',
 '=': '는',
 ':': '대',
 '~': '',
 '…': '',
 '△': '삼각형',
 '²': '의제곱',
 '②': '이번',
 '㈎': '가',
 '≤': '보다크거나같다',
 '㎝': '센치미터',
 '|': '절대값',
 '㈐': '다',
 '＝': '는',
 '÷': '나누기',
 '⑵': '이번',
 '㎠': '제곱센치미터',
 '¾': '사분의삼',
 '∠': '각',
 '%': '퍼센트',
 '－': '마이너스',
 "'": '',
 '③': '삼번',
 '＋': '더하기',
 '⑴': '일번',
 '√': '루트',
 'cos': '코사인',
 'sin': '사인',
 'tan': '탄젠트'}

In [261]:
ad = {
    'A':'에이',
    'B':'비',
    'C':'씨',
    'D':'디',
    'E':'이',
    'F':'에프',
    'G':'지',
    'H':'에이치',
    'I':'아이',
    'J':'제이',
    'K':'케이',
    'L':'엘',
    'M':'엠',
    'N':'엔',
    'O':'오',
    'P':'피',
    'Q':'큐',
    'R':'알',
    'S':'에스',
    'T':'티',
    'U':'유',
    'V':'브이',
    'W':'더블유',
    'X':'엑스',
    'Y':'와이',
    'Z':'지'
}

In [ ]:
s = '이건 우리가 중학교 2학년 때 배웠죠, 그렇죠? ∠A가 공통이고, 그다음에 둘 다 직각을 갖고 있으니까 AA닮음이 될 테고.'

In [210]:
def get_korean_pronunciation_of_number(n):
    nbased = {0 : '', 1 : '일', 2 : '이', 3 : '삼', 4 : '사', 5 : '오', 6 : '육', 7 : '칠', 8 : '팔', 9 : '구' }
    nkopro = {}
    for _ in range(n+1):
        t = _//1000
        h = (_%1000)//100
        d = (_%100)//10
        o = _%10

        if _ == 0:
            nkopro[_] = '영'
        else:
            tt = '' if t == 0 else ('천' if t == 1 else nbased[t] + '천')
            hh = '' if h == 0 else ('백' if h == 1 else nbased[h] + '백')
            dd = '' if d == 0 else ('십' if d == 1 else nbased[d] + '십')
            oo = '' if o == 0 else nbased[o]
            nkopro[_] = tt+hh+dd+oo
            
    return nkopro

In [211]:
# 초성 리스트. 00 ~ 18
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# 중성 리스트. 00 ~ 20
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

In [212]:
def korean_to_be_englished(korean_word):
    r_lst = []
    for w in list(korean_word.strip()):
        if '가'<=w<='힣':
            ch1 = (ord(w) - ord('가'))//588
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2
            r_lst.append([CHOSUNG_LIST[ch1], JUNGSUNG_LIST[ch2], JONGSUNG_LIST[ch3]])
        else:
            r_lst.append([w])
    return r_lst

In [213]:
#similarity
def similarity_prob(s1, s2):
    numerator =  0
    denominator = 3*len(s1)
    for _ in range(len(s1)):
        for c1, c2 in zip(s1[_], s2[_]):
            if c1 == c2:
                numerator += 1

    return numerator / denominator

In [265]:
with open(r'C:\Users\KYEONGCHAN LEE\(1-1.5) DataCampus\lec00001_EBS_linebyline.txt', 'r', encoding = 'utf-8') as f:
    elbl = f.read() #Ebs Line By Line
#     print(elbl)

반갑습니다, EBS 친구 여러분들. 여러분들과 함께 중학뉴런수학3(하)를 강의하게 된 손석민입니다.
2학년 뉴런을 통해서도 만났었는데요.
여러분들, 이렇게 다시 만나게 되어서 정말 반갑습니다.
앞으로 3(하) 과정 어떻게 진행되는지에 대해서는 여러분들께서 오리엔테이션 강의를 통해서 확인할 수 있을 거고요.
오리엔테이션 강의는 반드시 듣고 오셔야 여러분들이 3학년 과정이, (하) 과정이 특히 어떻게 수업되는지 확인할 수 있을 겁니다.
일단 3학년 과정 같은 경우에는 크게 3개의 대단원으로 이루어져 있는데요. 
첫 번째가 삼각비라는 단원이고 두 번째가 원의 성질, 세 번째가 통계. 이렇게 이루어져 있습니다.
우리가 1학년(하)도 마찬가지고 2학년(하)도 그래 왔듯, ‘주로 도형에 대한 이야기가 3(하)에서 많이 다루어지는구나.’ 라는 걸 생각하면 될 테고요.
오늘은 그 첫 시간이니까 ‘삼각비.’ ‘삼각’ 알아요. ‘비’ 알아요. 
근데 이제 합쳐져 있는 ‘삼각비’라는 새로운 용어가 나왔죠.
그럼 이 개념을 공부하기에 앞서서 당연히 삼각비의 뜻부터 시작하는 것이 첫 시간입니다.
오늘 그 첫 시간, 함께 시작해 보도록 할 텐데요. 중학교 3학년 수학! 손샘과 끝냅시다!
오늘 공부할 내용, 바로 두 가지 개념인데요. 오늘 두 가지 개념 첫 번째 제목이 굉장히 길어요. 
‘닮은 직각삼각형에서 대응변의 길이의 비’라는 것이 있습니다.
그다음 두 번째 개념이 바로 뭐냐면, ‘삼각비’라는 건데 삼각비의 뜻이겠죠? 
이 첫 번째 개념은, 앞으로 쭉 공부할 내용이 다 직각삼각형에서 주로 다루어질 텐데직각삼각형에서 갖는 기본적인 성질이 있는데 그 성질을 알고 가자는, ‘삼각비를 공부하기에 앞선 징검다리 내용이다.’ 라고 여러분들이 받아들이시면 됩니다.
그렇다면 ‘닮은 직각삼각형에서’라고 했으니까 닮은 직각삼각형을 그려 봐야 되겠죠? 
닮은 직각삼각형 두 개를 편하게 그릴 수 있는 제일 좋은 방법은 뭘까 생각해 보면우리는 이렇게 그리면 될 거 같아요. 
이렇게 한번 그려 보겠습니다,

In [266]:
with open(r'C:\Users\KYEONGCHAN LEE\(1-1.5) DataCampus\lec00001_STT1_Transcribe_linebyline.txt', 'r', encoding = 'utf-8') as f:
    glbl = f.read() #Ebs Line By Line
#     print(glbl)

반갑습니다. WITH 여러분들 여러분들과 함께 중학 유럽 수학 삼 하를 강행하기 된 분석입니다. 사. 
이 학년 늘어 를 통해서도 만났는데요. 
여러분들 이렇게 다시 만나게 되어서 정말 반갑습니다. 
앞으로 삼 하 과정 어떻게 진행되는지 에 대해서는 여러분들께서 오리엔테이션 강의를 통해서 확인할 수 있을 거고요.
 오리엔테이션 강의는 반드시 듣고 옷 여야 여러분들이 삼 학년 과정의 한 가정의 특히 어떻게 수업 되는지 확인할 수 있을 겁니다. 
기자 일단 삼 학년 과정 같은 경우에는 크게 세계의 대단원의 으로 이뤄져 있는데요.
 첫번째가 삼각 이라는 단원고 두 번째가 원에 성질, 세 번째가 통계 이렇게 이뤄져 있습니다. 
우리가 일 학년 하도 마찬가지고 이 학년 하도 그랬듯 주로 도용에 대한 이야기가 삼 하에서 많이 다뤄지는 구나 라는 걸 생각하면 될 테고요.
 오늘은 그 첫 시간이 이니까 삼 갑 의 삼각 알아요 이 알아요. 
합쳐져 있는 삼각 이라는 새로운 용어가 나왔죠. 
이 개념을 공부하게 앞서 당연히 삼각 비 듯 부터 시작하는 것이 첫 시간입니다. 
오늘 사 시간 함께 시작해 보도록 할 텐데요. 중학교 삼, 학년 수학 선생과 끝납니다. 
오늘 공부할 내용, 바로 두 가지 개념 인데요. 아! 오늘 두 가지 기념 첫 번째 제복이 굉장히 길어야 
닮은 즉각 삼 각 통에서 대응 변의 길이의 이 라는 것이 있습니다. 
그 다음 두 번째 개념이 바로 뭐냐하면 삼 갑 의 라는 건데 상납 뜻이겠죠 
의 첫 번째 개념은 두 번째 앞으로 쭉 공부할 내용이 다 즉각 삼각형 에서 주로 다뤄질 텐데 즉각 상황 평에서 갖는 기본적인 성질이 있는데 그 성질을 알고 가자 라는 어떤 삼각 비를 공부하기 앞선 징검다리 내용이라고 여러분들이 받아들이지만 됩니다. 
그렇다면 닮은 즉각 삼각형 해서 있으니까 닮은 즉각 삼각형을 그려 봐야겠죠. 
닮은 즉각 삼각형 두 개를 편하게 그릴 수 있는 제일 좋은 방법은 뭘까? 생각해보면 우리는 이렇게 그리면 될 것 같아요. 
이렇게 만들어보겠습니다 이

In [304]:
def get_similarity_table(text1, text2):
    pat_math_symbol = re.compile(r'[().,‘’?!]')
    pat_number = re.compile(r'\d+')
    kopro = get_korean_pronunciation_of_number(9999) #~9999
    
    esl = text1.splitlines() #문장단위로 리스트
    gsl = text2.splitlines()
    
    # punctuation 제거, 앞뒤 공백 제거
    new_esl = [s.strip() for s in [pat_math_symbol.sub('', _) for _ in esl]]
    new_gsl = [s.strip() for s in [pat_math_symbol.sub('', _) for _ in gsl]]
    
    #수학기호 한국어로 변환(EBS)
    for i in range(len(new_esl)):
        for j in range(len(new_esl[i])):
            s = new_esl[i][j]
            if s in sd:
                new_esl[i] = new_esl[i].replace(s, sd[s])
    
    #숫자 한국어로 변환(EBS)
    new_esl2 = []
    for _ in new_esl:
        fl = re.findall(r'\d+', _)
        for n in fl:
            _ = _.replace(n, kopro[int(n)])
        new_esl2.append(_)
        
    #숫자 한국어로 변환(STT)
    new_gsl2 = []
    for _ in new_gsl:
        fl = re.findall(r'\d+', _)
        for n in fl:
            _ = _.replace(n, kopro[int(n)])
        new_gsl2.append(_)
    
    #알파벳 한국어로 변환(EBS)
    for i in range(len(new_esl2)):
        for j in range(len(new_esl2[i])):
            s = new_esl2[i][j]
            if s in ad:
                new_esl2[i] = new_esl2[i].replace(s, ad[s])
                
    #알파벳 한국어로 변환(STT)
    for i in range(len(new_gsl2)):
        for j in range(len(new_gsl2[i])):
            s = new_gsl2[i][j]
            if s in ad:
                new_gsl2[i] = new_gsl2[i].replace(s, ad[s])
    
    if len(new_esl2) != len(new_gsl2):
        return print('error : not matched line number')
    
    
    maxprb_list_for_plot = []
    for _ in range(len(esl)):
        tokenized_ebs_sentence_list = TweetTokenizer().tokenize(new_esl2[_])
        serial_gsl_string = ''.join(TweetTokenizer().tokenize(new_gsl2[_]))
        print(new_esl2[_])
        print(new_gsl2[_])
        print()
        
        for word_index in range(len(tokenized_ebs_sentence_list)):
            word = tokenized_ebs_sentence_list[word_index]
            max_sim_prob = 0
            for compare_index in range(len(serial_gsl_string)-len(word)+1):
                s1 = korean_to_be_englished(word)
                compared_word = serial_gsl_string[compare_index:compare_index+len(word)]
                s2 = korean_to_be_englished(compared_word)
                prob = similarity_prob(s1, s2)
                if prob > max_sim_prob:
                    max_sim_prob = prob
                    max_sim_word = compared_word
            print('EBS 단어  :', word)
            print('GoogleSTT :', max_sim_word, max_sim_prob)
            print()
            maxprb_list_for_plot.append(max_sim_prob)
            similarity_table.append({word:max_sim_word}, ignore_index=True)
        print('----------------------------------')
    
    return similarity_table, maxprb_list_for_plot
    

In [305]:
table, maxprob_list = get_similarity_table(elbl, glbl)

반갑습니다 이비에스 친구 여러분들 여러분들과 함께 중학뉴런수학삼하를 강의하게 된 손석민입니다
반갑습니다 더블유아이티에이치 여러분들 여러분들과 함께 중학 유럽 수학 삼 하를 강행하기 된 분석입니다 사

EBS 단어  : 반갑습니다
GoogleSTT : 반갑습니다 1.0

EBS 단어  : 이비에스
GoogleSTT : 이티에이 0.75

EBS 단어  : 친구
GoogleSTT : 치여 0.5

EBS 단어  : 여러분들
GoogleSTT : 여러분들 1.0

EBS 단어  : 여러분들과
GoogleSTT : 여러분들과 1.0

EBS 단어  : 함께
GoogleSTT : 함께 1.0

EBS 단어  : 중학뉴런수학삼하를
GoogleSTT : 중학유럽수학삼하를 0.9259259259259259

EBS 단어  : 강의하게
GoogleSTT : 강행하기 0.6666666666666666

EBS 단어  : 된
GoogleSTT : 된 1.0

EBS 단어  : 손석민입니다
GoogleSTT : 된분석입니다 0.5555555555555556

----------------------------------
이학년 뉴런을 통해서도 만났었는데요
이 학년 늘어 를 통해서도 만났는데요

EBS 단어  : 이학년
GoogleSTT : 이학년 1.0

EBS 단어  : 뉴런을
GoogleSTT : 늘어를 0.4444444444444444

EBS 단어  : 통해서도
GoogleSTT : 통해서도 1.0

EBS 단어  : 만났었는데요
GoogleSTT : 도만났는데요 0.6111111111111112

----------------------------------
여러분들 이렇게 다시 만나게 되어서 정말 반갑습니다
여러분들 이렇게 다시 만나게 되어서 정말 반갑습니다

EBS 단어  : 여러분들
GoogleSTT : 여러분들 1.0

EBS 단어  : 이렇게
GoogleSTT : 이렇게 1.0

EBS 단어  : 다시
GoogleSTT : 다시 1.0

EBS 단어  : 만나게


EBS 단어  : 건데
GoogleSTT : 건데 1.0

EBS 단어  : 삼각비의
GoogleSTT : 삼갑의라 0.5833333333333334

EBS 단어  : 뜻이겠죠
GoogleSTT : 뜻이겠죠 1.0

----------------------------------
이 첫 번째 개념은 앞으로 쭉 공부할 내용이 다 직각삼각형에서 주로 다루어질 텐데직각삼각형에서 갖는 기본적인 성질이 있는데 그 성질을 알고 가자는 삼각비를 공부하기에 앞선 징검다리 내용이다 라고 여러분들이 받아들이시면 됩니다
의 첫 번째 개념은 두 번째 앞으로 쭉 공부할 내용이 다 즉각 삼각형 에서 주로 다뤄질 텐데 즉각 상황 평에서 갖는 기본적인 성질이 있는데 그 성질을 알고 가자 라는 어떤 삼각 비를 공부하기 앞선 징검다리 내용이라고 여러분들이 받아들이지만 됩니다

EBS 단어  : 이
GoogleSTT : 이 1.0

EBS 단어  : 첫
GoogleSTT : 첫 1.0

EBS 단어  : 번째
GoogleSTT : 번째 1.0

EBS 단어  : 개념은
GoogleSTT : 개념은 1.0

EBS 단어  : 앞으로
GoogleSTT : 앞으로 1.0

EBS 단어  : 쭉
GoogleSTT : 쭉 1.0

EBS 단어  : 공부할
GoogleSTT : 공부할 1.0

EBS 단어  : 내용이
GoogleSTT : 내용이 1.0

EBS 단어  : 다
GoogleSTT : 다 1.0

EBS 단어  : 직각삼각형에서
GoogleSTT : 즉각삼각형에서 0.9523809523809523

EBS 단어  : 주로
GoogleSTT : 주로 1.0

EBS 단어  : 다루어질
GoogleSTT : 데그성질 0.5833333333333334

EBS 단어  : 텐데직각삼각형에서
GoogleSTT : 이다즉각삼각형에서 0.8148148148148148

EBS 단어  : 갖는
GoogleSTT : 갖는 1.0

EBS 단어  : 기본적인
GoogleSTT : 기본적인 1.0

EBS 단어

EBS 단어  : 이대일
GoogleSTT : 이대일 1.0

EBS 단어  : 얼마
GoogleSTT : 얼마 1.0

EBS 단어  : 삼이
GoogleSTT : 삶의 0.6666666666666666

EBS 단어  : 돼야
GoogleSTT : 돼야 1.0

EBS 단어  : 된다는
GoogleSTT : 한다는 0.7777777777777778

EBS 단어  : 걸
GoogleSTT : 걸 1.0

EBS 단어  : 여러분들이
GoogleSTT : 여러분이할 0.6

EBS 단어  : 알
GoogleSTT : 일 0.6666666666666666

EBS 단어  : 수
GoogleSTT : 수 1.0

EBS 단어  : 있죠
GoogleSTT : 있죠 1.0

----------------------------------
그래서 우리는 닮음비라는 건 작은 삼각형과 큰 삼각형의 대응하는 변의 길이의 비이니까 오대십 삼대육 사대팔 이것이 모두 다 일대이로 같고 그것을 닮음비라고 한다
그래서 우리는 닮은 이라는 건 작은 상납 형과 큰 삼각형 에 대응하는 별이 길이 보니까 오 대 십 선대 육 사 대 팔 이것이 모두 다 일 대 이 로 갖고 그것을 닮은 이라고 한다

EBS 단어  : 그래서
GoogleSTT : 그래서 1.0

EBS 단어  : 우리는
GoogleSTT : 우리는 1.0

EBS 단어  : 닮음비라는
GoogleSTT : 닮은이라는 0.8666666666666667

EBS 단어  : 건
GoogleSTT : 건 1.0

EBS 단어  : 작은
GoogleSTT : 작은 1.0

EBS 단어  : 삼각형과
GoogleSTT : 삼각형에 0.8333333333333334

EBS 단어  : 큰
GoogleSTT : 큰 1.0

EBS 단어  : 삼각형의
GoogleSTT : 삼각형에 0.9166666666666666

EBS 단어  : 대응하는
GoogleSTT : 대응하는 1.0

EBS 단어  : 변의
GoogleSTT : 별이 0.66666

In [326]:
m = np.array(maxprob_list)

In [327]:
m = pd.Series(m)
m

0      1.000000
1      0.750000
2      0.500000
3      1.000000
4      1.000000
         ...   
465    1.000000
466    0.666667
467    1.000000
468    0.833333
469    0.416667
Length: 470, dtype: float64

In [335]:
data = pd.DataFrame(m.value_counts())

In [336]:
data.reset_index(level=0, inplace=True)

In [340]:
data

,index,0
0,1.000000,323
1,0.666667,45
2,0.833333,14
3,0.777778,13
4,0.500000,11
5,0.888889,10
6,0.444444,6
7,0.933333,6
8,0.866667,5
9,0.583333,4


In [344]:
data.sort_values(by='index')

,index,0
19,0.222222,1
10,0.333333,4
16,0.388889,1
14,0.416667,2
6,0.444444,6
4,0.500000,11
20,0.518519,1
12,0.555556,4
9,0.583333,4
29,0.600000,1


In [ ]:
s1 = '오분의삼이죠'
s2 = '오분의삼'

In [ ]:
s1 = '공부를 했었는데'
s2 = '공부했는데'

In [ ]:
s1 = '각에이가'
s2 = '가계가'